# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

#### --> Import Statements

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Datastore
from azureml.core.dataset import Dataset

from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails
from dataprocess import clean_data

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

In [3]:
cpu_cluster_name = "cpu-cluster2"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset consist information of patient that might be having heart disease. The target column has values 0 and 1 which determines if the patients has heart disease.
There are 14 attributes in the dataset as follows:
1. age
2. sex
3. chest pain type
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
14. target column, 0 = heart disease not present, 1 = heart disease present

#### --> Dataset Object

In [4]:
key = "heart-dataset"
description_text = "Heart Disease dataset"

#-- Dataset Call
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


## Preprocessing Data

After data is pulled, the data is processed for training in the following way:
- Standardization was performed on the column bloodpressure, thalac and cholestrol to save on computational cost.
- Rows with null values were dropped
- The age column was binned into 7 groups of '29-37', '38-45', '46-53', '54-61', '62-69' and '70-77'

After processing the data, the test input output and train input output data was set and fed to training.

In [5]:
x_df, y_df = clean_data(df)

In [6]:
### Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(x_df, y_df, test_size=0.30)
df_train = pd.concat([X_train,Y_train], axis=1)

#-- Train Data
ds_train = Dataset.Tabular.register_pandas_dataframe(dataframe= df_train, target= Datastore.get_default(ws), name="train_data")

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/e45d10a6-4b00-43ef-9491-833d0648b280/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The automl settings is configured as follows:
- Experiment timeout minutes determines how long experiment should run. In this case, its 20 minutes the experiment will run.
- Max concurrent iterations determines how many iterations should be running in parallel. We chose 5 iterations for this experiement.
- The primary metric to determine the performance of the runs being used here is AUC weighted.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'AUC_weighted'
}

The automl config is configured as follows:
- The compute target is set to the one created
- The task is classification as the target column has values 0 and 1
- Early stopping is enabled
- The path and log path is assigned

In [8]:
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = 'classification',
                             training_data = ds_train, #train data
                             label_column_name = "target", #target column with 0 & 1
                             path = './pipeline-project3',
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )

In [9]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_f342fd17-9b1a-4e29-84b6-8ff43af56146,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
#-- Use Run Details to show all accuracy

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_f342fd17-9b1a-4e29-84b6-8ff43af56146,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

In [11]:
remote_run.get_output()

(Run(Experiment: automl-exp,
 Id: AutoML_2080453b-f05a-4417-a3b9-e9d1810ad543_43,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                   n_estimators=400,
                                                                                                   n_jobs=1,
                           

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
automl_best_run = remote_run.get_best_child()

In [40]:
best_run_metrics = automl_best_run.get_metrics()
best_run_parameters = automl_best_run.get_details()
best_run_properties = best_run_parameters['properties']

#print(best_run_properties)
print(best_run_properties['run_algorithm'])
print(best_run_properties['score'])


VotingEnsemble
0.9389044289044289


In [23]:
#--- Save the best model
amodel= automl_best_run.register_model(model_name='udacity-automl-project', model_path='./outputs/model.pkl')

In [24]:
#--- Print the Best Model 
amodel

Model(workspace=Workspace.create(name='quick-starts-ws-143160', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-143160'), name=udacity-automl-project, id=udacity-automl-project:1, version=1, tags={}, properties={})

In [25]:
#--- Print last model from run
temp_model = remote_run.get_output()
temp_model[-1]

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_lambda=0.625,
                                                                                               scale_pos_weight=1,
                                                                                               seed=None,
                                                          

## Model Deployment Using ACIService

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [26]:
from azureml.core import Environment
import sklearn
from azureml.core.conda_dependencies import CondaDependencies

#--- Creating conda environment for the service
environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'azureml-automl-core',
    'azureml-automl-runtime',
    'packaging',
    'xgboost==0.90',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [27]:
with open('score.py') as f:
    print(f.read())

import joblib
import numpy as np
import os
import pickle
import json
import pandas as pd



# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'model.pkl'

    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)
    print("Found model:",os.path.isfile(model_path))
    
    print(model_path)

    model = joblib.load(model_path)


# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and validates the incoming payload against
# the example

In [31]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Model

service_name = 'my-custom-env-service12'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,  auth_enabled = True, description = 'AutoML model for Heart Disease data')
#local_config = LocalWebservice.deploy_configuration(port=6788) #-- local webservice for testing

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[amodel],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-19 17:20:29+00:00 Creating Container Registry if not exists.
2021-04-19 17:20:29+00:00 Registering the environment.
2021-04-19 17:20:31+00:00 Building image..
2021-04-19 17:35:36+00:00 Generating deployment configuration.
2021-04-19 17:35:37+00:00 Submitting deployment to compute..
2021-04-19 17:35:42+00:00 Checking the status of deployment my-custom-env-service12..
2021-04-19 17:38:23+00:00 Checking the status of inference endpoint my-custom-env-service12.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [32]:
print(service.get_logs())

2021-04-19T17:38:13,378547600+00:00 - iot-server/run 
2021-04-19T17:38:13,389422200+00:00 - nginx/run 
2021-04-19T17:38:13,388300500+00:00 - gunicorn/run 
2021-04-19T17:38:13,380178400+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

TODO: In the cell below, send a request to the web service you deployed to test it.

In [33]:
#--  Making webservice call using service.run()
import json

input_payload = json.dumps({
    'data': X_test.to_dict(),
    'method': 'predict_proba'
})

output = service.run(input_payload)
print(output)

[0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1]


In [34]:
print(output)

[0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1]


In [38]:
#-- Making webservice call using RestEndpoint URI key

import requests
key1, key2 = service.get_keys()
#key1 = 'http://360cff5b-18ca-4395-b44b-36da2dd3a43c.southcentralus.azurecontainer.io/score'

input_data = json.dumps({'data':X_test[0:1].to_dict()})
print(input_data)

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)

{"data": {"age_bins": {"242": 5}, "sex": {"242": 1}, "cp": {"242": 0}, "trestbps": {"242": 0.7639557707131766}, "chol": {"242": -0.6621688357951817}, "fbs": {"242": 0}, "restecg": {"242": 0}, "thalach": {"242": -0.7717063030099474}, "exang": {"242": 0}, "oldpeak": {"242": 2.0}, "slope": {"242": 1}, "ca": {"242": 2}, "thal": {"242": 1}}}
POST to url http://e425d6b5-dc12-45b4-9cf8-a6ed91275fd5.southcentralus.azurecontainer.io/score
prediction: [0]


TODO: In the cell below, print the logs of the web service and delete the service

In [36]:
print(service.get_logs())

2021-04-19T17:38:13,378547600+00:00 - iot-server/run 
2021-04-19T17:38:13,389422200+00:00 - nginx/run 
2021-04-19T17:38:13,388300500+00:00 - gunicorn/run 
2021-04-19T17:38:13,380178400+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7e29e14852b8e63cc438037e59324eb8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [41]:
service.delete()